# **SimpleChain**

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import GPT4AllEmbeddings

In [15]:
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from langchain_community.llms import CTransformers
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

# **Embedding and Store vector DB**

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import GPT4AllEmbeddings

In [20]:
# Khai bao bien
data_path = "data"
vector_db_path = "vectorstores"

In [21]:
def create_db_from_text():
  text = """LLMs have become a household name thanks to the role they have played
  in bringing generative AI to the forefront of the public interest, as well as
  the point on which organizations are focusing to adopt artificial intelligence
  across numerous business functions and use cases.
  Outside of the enterprise context, it may seem like LLMs have arrived out of the
  blue along with new developments in generative AI. However, many companies, including
  IBM, have spent years implementing LLMs at different levels to enhance their natural
  language understanding (NLU) and natural language processing (NLP) capabilities.
  This has occurred alongside advances in machine learning, machine learning models,
  algorithms, neural networks and the transformer models that provide the architecture for these AI systems.
  LLMs are a class of foundation models, which are trained on enormous amounts of
  data to provide the foundational capabilities needed to drive multiple use cases
  and applications, as well as resolve a multitude of tasks. This is in stark contrast
  to the idea of building and training domain specific models for each of these use
  cases individually, which is prohibitive under many criteria (most importantly cost
  and infrastructure), stifles synergies and can even lead to inferior performance."""

  splitter = CharacterTextSplitter(chunk_size=500,
                                   chunk_overlap=50,
                                   )
  chunks = splitter.split_text(text)

  #Embedding
  embedd_model = "all-MiniLM-L6-v2.gguf2.f16.gguf"
  gpt4all_kwargs = {'allow_download': 'True'}

  embeddings = GPT4AllEmbeddings(
    model_name=embedd_model,
    gpt4all_kwargs=gpt4all_kwargs
  )

  #Store vector DB into FAISS
  db = FAISS.from_texts(chunks, embeddings)
  db.save_local(vector_db_path)

  return db

def create_db_from_files(data_path):
  loader = DirectoryLoader(data_path, glob="**/*.pdf", loader_cls = PyPDFLoader)
  documents = loader.load()

  splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  documents = splitter.split_documents(documents)

  #Embeddings
  embedd_model = "all-MiniLM-L6-v2.gguf2.f16.gguf"
  gpt4all_kwargs = {'allow_download': 'True'}

  embeddings = GPT4AllEmbeddings(
    model_name=embedd_model,
    gpt4all_kwargs=gpt4all_kwargs
  )
  db = FAISS.from_documents(documents, embeddings)
  db.save_local(vector_db_path)

  return db



In [ ]:
create_db_from_files(data_path)


# **Retriver**

In [8]:
from langchain_community.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline


In [9]:
def load_llm(model_name):
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )
    llm = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        low_cpu_mem_usage=True
    )

    return llm

# Create QA chain
def create_qa_chain(prompt, llm, db):
    llm_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=db.as_retriever(search_kwargs={"k": 3}, max_tokens_limit=1024),
        return_source_documents=False,
        chain_type_kwargs={'prompt': prompt}
    )
    return llm_chain

# Load vector DB
def load_vector_db(vector_db_path):
    # Embedding
    model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"
    gpt4all_kwargs = {'allow_download': 'True'}

    embeddings = GPT4AllEmbeddings(
      model_name=model_name,
      gpt4all_kwargs=gpt4all_kwargs
    )

    # Allow dangerous deserialization
    db = FAISS.load_local(vector_db_path, embeddings=embeddings, allow_dangerous_deserialization=True)
    return db


In [ ]:
# Cau hinh
model_file = "uonlp/Vistral-7B-Chat-gguf"
vector_db_path = "./vectorstores"

db = load_vector_db(vector_db_path)
llm = load_llm(model_file)

In [ ]:
template = '''<s>[INST] <<SYS>>
Bạn là một trợ lí Tiếng Việt nhiệt tình và trung thực. Hãy luôn trả lời một cách hữu ích nhất có thể, đồng thời giữ an toàn.
Câu trả lời của bạn không nên chứa bất kỳ nội dung gây hại, phân biệt chủng tộc, phân biệt giới tính, độc hại, nguy hiểm hoặc bất hợp pháp nào. Hãy đảm bảo rằng các câu trả lời của bạn không có thiên kiến xã hội và mang tính tích cực.Nếu một câu hỏi không có ý nghĩa hoặc không hợp lý về mặt thông tin, hãy giải thích tại sao thay vì trả lời một điều gì đó không chính xác. Nếu bạn không biết câu trả lời cho một câu hỏi, hãy trẳ lời là bạn không biết và vui lòng không chia sẻ thông tin sai lệch.
<</SYS>>

{prompt} [/INST]'''


prompt = PromptTemplate(template=template, input_variables=["prompt"])

In [ ]:
chain_type_kwargs = {"prompt": prompt}

In [ ]:
retriever = db.as_retriever(search_kwargs={"k": 5})

In [ ]:

user_input = "Hãy cho tôi biết về thủ đô của Việt Nam."
 # Format the prompt using the template
formatted_prompt = prompt.format(prompt=user_input)

qa_chain = create_qa_chain(formatted_prompt, llm, db)


In [ ]:
# Example usage of the QA chain
context = "I am a senior student at TDT University, I am majoring in Computer Science"
question = "Are you a student?"

response = qa_chain({"context": context, "query": question})
print(response)